In [1]:
from koselleck import *

In [2]:
swas=get_word_abstractness_scores()

In [3]:
# def prdz(y,ystart=1710,yend=3000,ystep=40):
def prdz(y,ystart=1600,yend=3000,ystep=20):
    ln=None
    for n in range(ystart,yend,ystep):
        if ln is None: ln=n
        if y<n: return ln
        ln=n
        
prdz(1715)

1700

In [4]:
def get_neighb_data(ystart=1600,ystep=20):
    df=pd.read_pickle('data/data.neighborhoods.cache.v2.pkl')
    df['period_int']=df.period.apply(lambda x:int(x[:4]))
    df['period2']=df.period_int.apply(lambda y: prdz(y,ystart=ystart,ystep=ystep))
    #df['period']=df['period']
    return df#.query('1700<=period_int<1900')


dfres=df=get_neighb_data()
df.groupby('period').size()
df

,period,word2,rank,count,rankcount,rank2,period_int,period2
word,,,,,,,,
abandonment,1780-1790,insuperable,17.25,4,0.219178,61.0,1780,1780
abandonment,1780-1790,exactions,2.00,1,0.333333,27.0,1780,1780
abandonment,1780-1790,friary,3.00,1,0.250000,49.0,1780,1780
abandonment,1780-1790,indemnification,34.00,4,0.114286,137.5,1780,1780
abandonment,1780-1790,uncandid,5.00,1,0.166667,100.5,1780,1780
...,...,...,...,...,...,...,...,...
zoology,1950-1960,morel,39.00,1,0.025000,403.5,1950,1940
zoology,1950-1960,limbless,41.00,1,0.023810,411.5,1950,1940
zoology,1950-1960,stabs,42.00,1,0.023256,418.5,1950,1940


In [5]:
dfabs=get_dfpiv_abs().reset_index().melt(id_vars=['word'],value_name='abs')
dfabs['period2']=dfabs['period'].apply(prdz)
dfabs=dfabs.groupby(['word','period2']).mean().reset_index().dropna()
dfabs['word2']=dfabs['word']
okwords=set(dfabs.word)
dfabs

,word,period2,abs,word2
4,abandonment,1780,1.249668,abandonment
5,abandonment,1800,2.563577,abandonment
6,abandonment,1820,2.896265,abandonment
7,abandonment,1840,2.677320,abandonment
8,abandonment,1860,2.750508,abandonment
...,...,...,...,...
60005,zoology,1800,0.090642,zoology
60006,zoology,1820,-0.096660,zoology
60007,zoology,1840,0.144612,zoology
60008,zoology,1860,0.079964,zoology


In [6]:
def show(
        w='culture',
        min_periods=2,
        max_rank=20,
        max_rank1=10,
        width=8,
        height=6,
        save=False,
        vnum='v13-20y',
        showdata=False,
        year_min=1720,
        year_max=1900):
    
    if w in set(dfres.index):
        wdf=dfres.loc[w].query(f'period2>={year_min}')
        wdf=dfres.loc[w].query(f'period2<{year_max}')
        wdf=wdf[wdf.word2.isin(okwords)]
        wdf=wdf[~wdf.word2.isin({'foil','slate'})]
        figdf=wdf
        words_ever_high=set(wdf[wdf.rank2<=max_rank1].word2)
        figdf=figdf[figdf.word2.isin(words_ever_high)].groupby(['word2','period2']).mean().reset_index()
        wdfp=figdf.pivot('word2','period2','rankcount')
        figdf=wdfp.reset_index().melt(id_vars=['word2'],value_name='rankcount')
        figdf=figdf.groupby('word2').filter(lambda gdf: len(gdf.dropna())>=min_periods)
        figdf=pd.concat(
            grp.assign(rank3=(-1*grp['rankcount']).rank(method='first'))
            for i,grp in figdf.groupby('period2')
        )
        figdf=figdf[figdf.rank3<max_rank]
        figdf=figdf.sort_values('rankcount')#.set_index('word2')
#         figdf=figdf.merge(dfabs,on=['word2','period2'])
#         figdf['conc']=figdf['abs']*-1
#         figdf['is_abs']=figdf['abs']>0
        figdf['period_str']=[
            f'{x}' #'-{x+40 if x+40<1900 else 1900}'
            for x in figdf['period2']
        ]
        figdf['word']=figdf['word2']  
        if showdata: display(figdf)
        fig=start_fig(
            figdf,
            x='period_str',
            y='rank3',
            label='word',
            group='word',
#             fill='conc',
            figure_size=(width,height)
        ) + p9.geom_line(size=0.5) + p9.scale_y_reverse()
        
        first_appearances=figdf.sort_values('period_str').drop_duplicates('word')
        other_appearances=figdf.loc[set(figdf.index) - set(first_appearances.index)]
        
        fig+=p9.geom_label(size=6.5,fill='#BBBBBB',fontweight='bold',data=first_appearances)
        fig+=p9.geom_label(size=6.5,data=other_appearances)
        
        fig+=p9.scale_fill_gradient(low='white',high='#999999')
        fig+=p9.scale_color_gradient(low='white',high='#999999')
        fig+=p9.labs(
            title=f'Changing associations of "{w}"',
            x='Date of semantic model',
            y=f'Proximity to "{w}" →',
            fill='Concreteness'
        )
        if save:
            ofn=f'fig.changing_simple.{w}.{vnum}.png'
            fig.save(f'figures/{ofn}')
            fig.save(f'/home/ryan/Markdown/Drafts/TheGreatAbstraction/figures/{ofn}')
        display(fig)
        return figdf
    else:
        print(f'!! {w} not in data')
    
    
res=interact_manual(show, w='progress')

interactive(children=(Text(value='progress', description='w'), IntSlider(value=2, description='min_periods', m…